In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import numpy as np
import time
import os
import copy
from PIL import Image


# Load Dataset

In [ ]:
!git clone https://github.com/tmilitino/pokedex.git 

## Split dataset

In [ ]:
import shutil
import os
from sklearn.model_selection import train_test_split

## 'Bulbasaur','Charmander','Squirtle'

In [ ]:
base_dir = '/content/pokedex'
base_dir_train = '/content/pokedex/train'
base_dir_valid = '/content/pokedex/valid'
os.makedirs(base_dir_train)
os.makedirs(base_dir_valid)

In [ ]:
for pokemon in ['Bulbasaur','Charmander','Squirtle']:
  pokemon_files = os.listdir(os.path.join(base_dir,'pokedex_alfa',pokemon))
  train, valid = train_test_split(pokemon_files, test_size=0.2, random_state=42)
  os.makedirs(os.path.join(base_dir_train,pokemon))
  os.makedirs(os.path.join(base_dir_valid,pokemon))
  for archive in train:
    shutil.move(os.path.join(base_dir,'pokedex_alfa',pokemon,archive), os.path.join(base_dir_train,pokemon,archive))
  for archive in valid:
    shutil.move(os.path.join(base_dir,'pokedex_alfa',pokemon,archive), os.path.join(base_dir_valid,pokemon,archive))


## Evolução

In [ ]:
base_dir_evo = '/content/pokedex/evolucao'
base_dir_evo_train = '/content/pokedex/evolucao/train'
base_dir_evo_valid = '/content/pokedex/evolucao/valid'
os.makedirs(base_dir_evo_train)
os.makedirs(base_dir_evo_valid)

In [ ]:
for pokemon in ['Pikachu','Raichu']:
  pokemon_files = os.listdir(os.path.join(base_dir_evo,pokemon))
  train, valid = train_test_split(pokemon_files, test_size=0.2, random_state=1)
  os.makedirs(os.path.join(base_dir_evo_train,pokemon))
  os.makedirs(os.path.join(base_dir_evo_valid,pokemon))
  for archive in train:
    shutil.move(os.path.join(base_dir_evo,pokemon,archive), os.path.join(base_dir_evo_train,pokemon,archive))
  for archive in valid:
    shutil.move(os.path.join(base_dir_evo,pokemon,archive), os.path.join(base_dir_evo_valid,pokemon,archive))

# Experimentos

### Funções Basicas

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            current_loss = 0.0
            current_corrects = 0

            # Here's where the training happens
            print('Iterating through data...')

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # We need to zero the gradients, don't forget it
                optimizer.zero_grad()

                # Time to carry out the forward training poss
                # We only need to log the loss stats if we are in training phase
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # We want variables to hold the loss statistics
                current_loss += loss.item() * inputs.size(0)
                current_corrects += torch.sum(preds == labels.data)

            epoch_loss = current_loss / dataset_sizes[phase]
            epoch_acc = current_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # Make a copy of the model if the accuracy on the validation set has improved
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_since = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_since // 60, time_since % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # Now we'll load in the best model weights and return it
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_handeled = 0

    with torch.no_grad():
        for i, (inputs, labels ) in enumerate(dataloaders['valid']):

            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_handeled += 1
                fig = plt.figure()
                ax = fig.add_subplot(1, 2, 1)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_handeled == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)


In [ ]:
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([mean_nums])
    std = np.array([std_nums])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001) 

In [ ]:
def save_model(model, optimizer, save_path, epoch):
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch': epoch
    }, save_path)

In [ ]:
def load_model(model, optimizer, load_path):
    checkpoint = torch.load(load_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    
    return model, optimizer, epoch

In [ ]:
def predict(model, test_image_name, chosen_transforms):

    transform = chosen_transforms['valid']

    test_image = Image.open(test_image_name).convert('RGB')

    plt.imshow(test_image)

    test_image_tensor = transform(test_image)

    if torch.cuda.is_available():
        test_image_tensor = test_image_tensor.view(1, 3, 224, 224).cuda()
    else:
        test_image_tensor = test_image_tensor.view(1, 3, 224, 224)

    with torch.no_grad():
        model.eval()
        out = model(test_image_tensor)
        _, preds = torch.max(out, 1)

        print("Image class:  ", class_names[preds[0]])

In [ ]:
def predict_gray(model, test_image_name, chosen_transforms):

    transform = chosen_transforms['valid']

    test_image = Image.open(test_image_name).convert('RGB')

    plt.imshow(test_image)

    test_image_tensor = transform(test_image)

    if torch.cuda.is_available():
        test_image_tensor = test_image_tensor.view(1, 1, 224, 224).cuda()
    else:
        test_image_tensor = test_image_tensor.view(1, 1, 224, 224)

    with torch.no_grad():
        model.eval()
        out = model(test_image_tensor)
        _, preds = torch.max(out, 1)

        print("Image class:  ", class_names[preds[0]])

## Iniciais (Bulbasaur, Charmander, Squirtle)

### Resnet - 3 canais (RGB)

In [ ]:
mean_nums = [0.485, 0.456, 0.406]
std_nums = [0.229, 0.224, 0.225]

chosen_transforms = {'train': transforms.Compose([
        transforms.RandomResizedCrop(size=256),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean_nums, std_nums)
]), 'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean_nums, std_nums)
]),
}

In [ ]:
chosen_datasets = {x: datasets.ImageFolder(os.path.join(base_dir, x), chosen_transforms[x]) for x in ['train', 'valid']}

In [ ]:
chosen_datasets

In [ ]:
dataloaders = {x: torch.utils.data.DataLoader(chosen_datasets[x], batch_size=4,
  shuffle=True, num_workers=2)
              for x in ['train', 'valid']}

In [ ]:
dataset_sizes = {x: len(chosen_datasets[x]) for x in ['train', 'valid']}
class_names = chosen_datasets['train'].classes

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
print(dataset_sizes)
print(class_names)

In [ ]:
inputs, classes = next(iter(dataloaders['train']))

out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

In [ ]:
res_mod_rgb = models.resnet34(pretrained=True)

num_ftrs = res_mod_rgb.fc.in_features
res_mod_rgb.fc = nn.Linear(num_ftrs, 3)

In [ ]:
res_mod_rgb

In [ ]:
for name, child in res_mod_rgb.named_children():
    print(name)

In [ ]:
res_mod_rgb = res_mod_rgb.to(device)
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(res_mod_rgb.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
base_model_rgb = train_model(res_mod_rgb, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=3)
visualize_model(base_model_rgb)
plt.show()

In [ ]:
path_model = '/content/pokedex/pokedex_rgb.pt'

save_model(base_model_rgb, optimizer_ft, path_model, 3)

In [ ]:
res_mod_rgb = models.resnet34(pretrained=False)
num_ftrs = res_mod_rgb.fc.in_features
res_mod_rgb.fc = nn.Linear(num_ftrs, 3)

optimizer_ft = optim.SGD(res_mod_rgb.parameters(), lr=0.001, momentum=0.9)

model, optimizer, epoch = load_model(res_mod_rgb, optimizer_ft,  path_model)



In [ ]:
predict(model, '/content/pokedex/PokemonData/Metapod/00cc01c6418b4a268691127baf8cc8ce.jpg', chosen_transforms)

In [ ]:
predict(model, '/content/pokedex/PokemonData/Golduck/8eb4f82fad814eba820201e26199fc09.jpg', chosen_transforms)

### Resnet - 1 canal (Grayscale)

In [ ]:
mean_nums = [0.485, 0.456, 0.406]
std_nums = [0.229, 0.224, 0.225]

chosen_transforms = {'train': transforms.Compose([
        transforms.RandomResizedCrop(size=256),
        transforms.RandomRotation(degrees=15),
        transforms.Grayscale(),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        # transforms.Normalize(mean_nums, std_nums)
]), 'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.Grayscale(),
        transforms.ToTensor(),
        # transforms.Normalize(mean_nums, std_nums)
]),
}

In [ ]:
chosen_datasets = {x: datasets.ImageFolder(os.path.join(base_dir, x), chosen_transforms[x]) for x in ['train', 'valid']}

In [ ]:
chosen_datasets

In [ ]:
dataloaders = {x: torch.utils.data.DataLoader(chosen_datasets[x], batch_size=4,
  shuffle=True, num_workers=2)
              for x in ['train', 'valid']}

In [ ]:
dataset_sizes = {x: len(chosen_datasets[x]) for x in ['train', 'valid']}
class_names = chosen_datasets['train'].classes

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
print(dataset_sizes)
print(class_names)

In [ ]:
inputs, classes = next(iter(dataloaders['train']))

out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

In [ ]:
res_mod_gray = models.resnet34(pretrained=True)

num_ftrs = res_mod_gray.fc.in_features
res_mod_gray.fc = nn.Linear(num_ftrs, 3)

In [ ]:
res_mod_gray.conv1 = nn.Conv2d(1, 64, kernel_size=(5, 5), stride=(2, 2), padding=(3, 3), bias=False)

In [ ]:
res_mod_gray

In [ ]:
for name, child in res_mod_gray.named_children():
    print(name)

In [ ]:
res_mod_gray = res_mod_gray.to(device)
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(res_mod_gray.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
base_model_gray = train_model(res_mod_gray, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=10)
visualize_model(base_model_gray)
plt.show()

In [ ]:
path_model = '/content/pokedex/pokedex_gray.pt'

save_model(base_model_gray, optimizer_ft, path_model, 3)

In [ ]:
save_model(base_model_gray, optimizer_ft, path_model, 3)
res_mod_gray = models.resnet34(pretrained=False)
res_mod_gray.conv1 = nn.Conv2d(1, 64, kernel_size=(5, 5), stride=(2, 2), padding=(3, 3), bias=False)
num_ftrs = res_mod_gray.fc.in_features
res_mod_gray.fc = nn.Linear(num_ftrs, 3)

optimizer_ft = optim.SGD(res_mod_gray.parameters(), lr=0.001, momentum=0.9)

model, optimizer, epoch = load_model(res_mod_gray, optimizer_ft,  path_model)


In [ ]:
predict_gray(model, '/content/pokedex/PokemonData/Golduck/8eb4f82fad814eba820201e26199fc09.jpg', chosen_transforms)

In [ ]:
predict_gray(model, '/content/pokedex/PokemonData/Metapod/00cc01c6418b4a268691127baf8cc8ce.jpg', chosen_transforms)

## Evolução (Pikachu -> Raichu)

### Resnet - 3 canais (RGB)

In [ ]:
mean_nums = [0.485, 0.456, 0.406]
std_nums = [0.229, 0.224, 0.225]

chosen_transforms = {'train': transforms.Compose([
        transforms.RandomResizedCrop(size=256),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean_nums, std_nums)
]), 'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean_nums, std_nums)
]),
}

In [ ]:
chosen_datasets = {x: datasets.ImageFolder(os.path.join(base_dir_evo, x), chosen_transforms[x]) for x in ['train', 'valid']}

In [ ]:
chosen_datasets

In [ ]:
dataloaders = {x: torch.utils.data.DataLoader(chosen_datasets[x], batch_size=4,
  shuffle=True, num_workers=2)
              for x in ['train', 'valid']}

In [ ]:
dataset_sizes = {x: len(chosen_datasets[x]) for x in ['train', 'valid']}
class_names = chosen_datasets['train'].classes

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
print(dataset_sizes)
print(class_names)

In [ ]:
inputs, classes = next(iter(dataloaders['train']))

out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

In [ ]:
res_mod_evo_rgb = models.resnet34(pretrained=True)

num_ftrs = res_mod_evo_rgb.fc.in_features
res_mod_evo_rgb.fc = nn.Linear(num_ftrs, 2)

In [ ]:
res_mod_evo_rgb

In [ ]:
for name, child in res_mod_evo_rgb.named_children():
    print(name)

In [ ]:
res_mod_evo_rgb = res_mod_evo_rgb.to(device)
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(res_mod_evo_rgb.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
base_model_evo_rgb = train_model(res_mod_evo_rgb, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=5)
visualize_model(base_model_evo_rgb)
plt.show()

In [ ]:
path_model = '/content/pokedex/pokedex_evo_rgb.pt'

save_model(base_model_evo_rgb, optimizer_ft, path_model, 3)

In [ ]:
res_mod_evo_rgb = models.resnet34(pretrained=False)
num_ftrs = res_mod_evo_rgb.fc.in_features
res_mod_evo_rgb.fc = nn.Linear(num_ftrs, 2)

optimizer_ft = optim.SGD(res_mod_evo_rgb.parameters(), lr=0.001, momentum=0.9)

model, optimizer, epoch = load_model(res_mod_evo_rgb, optimizer_ft,  path_model)


In [ ]:
predict(model, '/content/pokedex/PokemonData/Charizard/001d0fa5713a4b01b6a8772370d9bf98.jpg', chosen_transforms)

### Resnet - 1 canal (Grayscale)

In [ ]:
mean_nums = [0.485, 0.456, 0.406]
std_nums = [0.229, 0.224, 0.225]

chosen_transforms = {'train': transforms.Compose([
        transforms.RandomResizedCrop(size=256),
        transforms.RandomRotation(degrees=15),
        transforms.Grayscale(),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
]), 'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.Grayscale(),
        transforms.ToTensor(),
]),
}

In [ ]:
chosen_datasets = {x: datasets.ImageFolder(os.path.join(base_dir_evo, x), chosen_transforms[x]) for x in ['train', 'valid']}

In [ ]:
chosen_datasets

In [ ]:
dataloaders = {x: torch.utils.data.DataLoader(chosen_datasets[x], batch_size=4,
  shuffle=True, num_workers=2)
              for x in ['train', 'valid']}

In [ ]:
dataset_sizes = {x: len(chosen_datasets[x]) for x in ['train', 'valid']}
class_names = chosen_datasets['train'].classes

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
print(dataset_sizes)
print(class_names)

In [ ]:
inputs, classes = next(iter(dataloaders['train']))

out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

In [ ]:
res_mod_evo_gray = models.resnet34(pretrained=True)

num_ftrs = res_mod_evo_gray.fc.in_features
res_mod_evo_gray.fc = nn.Linear(num_ftrs, 2)

In [ ]:
res_mod_evo_gray.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [ ]:
res_mod_evo_gray

In [ ]:
for name, child in res_mod_evo_gray.named_children():
    print(name)

In [ ]:
res_mod_evo_gray = res_mod_evo_gray.to(device)
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(res_mod_evo_gray.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
base_model_evo_gray = train_model(res_mod_evo_gray, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=10)
visualize_model(base_model_evo_gray)
plt.show()

In [ ]:
path_model = '/content/pokedex/pokedex_evo_gray.pt'

save_model(base_model_evo_gray, optimizer_ft, path_model, 3)

In [ ]:
res_mod_evo_gray = models.resnet34(pretrained=False)
res_mod_evo_gray.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
num_ftrs = res_mod_evo_gray.fc.in_features
res_mod_evo_gray.fc = nn.Linear(num_ftrs, 2)

optimizer_ft = optim.SGD(res_mod_evo_gray.parameters(), lr=0.001, momentum=0.9)

model, optimizer, epoch = load_model(res_mod_evo_gray, optimizer_ft,  path_model)


In [ ]:
predict_gray(model, '/content/pokedex/PokemonData/Golduck/8eb4f82fad814eba820201e26199fc09.jpg', chosen_transforms)

In [ ]:
predict_gray(model, '/content/pokedex/PokemonData/Metapod/00cc01c6418b4a268691127baf8cc8ce.jpg', chosen_transforms)